In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', None)

plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline
pd.set_option('display.max_columns', None)

In [104]:
data_string=r"C:\Users\am513\PS-88-21-DEV\lab\aaronfolder\NHIS2009_clean.dta" 
df_og=pd.read_stata(data_string, convert_missing=True)
df=pd.read_stata(data_string)
df_og;
#df.to_csv("NHIS2009_clean.csv")

In [191]:
df_og["hi_hsb1"]=pd.to_numeric(df_og["hi_hsb1"],errors="coerce")
df_og["hi"]=pd.to_numeric(df_og["hi"],errors="coerce")
df_og["age"]=pd.to_numeric(df_og["age"],errors="coerce")
#df_og.replace
#df_og["educ"]=pd.to_numeric(df_og["age"],errors="coerce")#why did this take so long :(

In [192]:
df_og2=df_og.query("marradult == 1 & perweight != 0")
df_og2;

In [193]:
df2=df.query("marradult == 1 & perweight != 0") #keep if marradult==1 & perweight!=0 
df2;

In [194]:
df_og3=df_og2.groupby("serial").sum().reset_index()[["serial","hi_hsb1"]]

In [195]:
df_og4=df_og2.merge(df_og3,how="left",on="serial")
df_og4;

In [196]:
df_og5=df_og4.drop("hi_hsb1_x",axis=1).rename(columns={"hi_hsb1_y":"hi_hsb"})

In [197]:
df_og5_5=df_og5[["serial","hi_hsb"]].set_index("serial")
df_og5_5;#.dtypes #by serial: egen hi_hsb = mean(hi_hsb1)

In [198]:
df_og6=df_og5.dropna(subset=["hi_hsb","hi"],how="any",axis="rows")
df_og6;

In [199]:
males=[
27076,
32501,
36537,
12722,
5958,
18162,
35909,
34396,
17814,
10369,
37415,
36924,
20036,
19066,
22639,
34048,
35904,
29606,
24936,
34048,
21939,
29126,
29542,
2605,
15837,
35956,
22639,
2605,
21666,
2265,
15837,
]

In [200]:
df_og6["female"]=1

In [201]:
df_og6.loc[df_og6["serial"].isin(males),"female"]=0 #by serial: egen female = total(fml)

In [202]:
df_og6["female"].value_counts()

1    29941
0       31
Name: female, dtype: int64

In [203]:
df_og7=df_og6.query("female == 1").drop("female",axis="columns") #keep if female==1     drop female
df_og7;

In [361]:
df_og7.dtypes

year               float32
inc1               float32
inc2               float32
inc3               float32
inc4               float32
inc5               float32
inc6               float32
inc7               float32
inc8               float32
serial               int32
hhweight             int32
pernum                int8
perweight          float64
sampweight         float64
age                float64
marstat           category
sex               category
famsize           category
relate            category
racenew           category
educ              category
educrec1          category
empstat           category
incfam07on        category
health            category
uninsured         category
age2               float32
fml                float32
nwhite             float32
hi                 float32
yedu                object
empl               float32
hlth               float32
inc                float32
incmp              float32
brooks             float32
marradult          float32
m

In [216]:
df_og8=df_og7.query("age>=26 & age<=59 & marradult==1 & adltempl>=1")

In [222]:
df_og9=df_og8.groupby(["serial"]).filter(lambda x: len(x)>1) #by serial: gen n = _N   keep if n>1 (This is also really bad code)

In [423]:
husband_somehi_ss=len(df_og9.query("hi == 1 & fml == 0"));
husband_nohi_ss=len(df_og9.query("hi == 0 & fml == 0"));
husband_somehi_age=np.round(np.mean(df_og9.query("hi == 1 & fml == 0")["age"]),decimals=2);
husband_nohi_age=np.round(np.mean(df_og9.query("hi == 0 & fml == 0")["age"]),decimals=2);
husband_diff_age=df_og9.query("hi == 0 & fml == 0")["age"].sem()-(df_og9.query("hi == 1 & fml == 0")["age"]).sem()
husband_somehi_edu=np.round(np.mean(df_og9.query("hi == 1 & fml == 0")["yedu"]),decimals=2)
husband_nohi_edu=np.round(np.mean(df_og9.query("hi == 0 & fml == 0")["yedu"]),decimals=2)
husband_diff_edu=np.round(df_og9.query("hi == 0 & fml == 0")["yedu"].sem()-(df_og9.query("hi == 1 & fml == 0")["yedu"]).sem(),decimals=2)
husband_diff_edu_diff=np.round(husband_somehi_edu-husband_nohi_edu,decimals=2)
df_og9["famsize num only"]=df_og9["famsize"].str.split(expand=True)[0]
df_og9["famsize num only"]=pd.to_numeric(df_og9["famsize num only"],errors="coerce")
husband_nohi_famsize=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["famsize num only"],dtype=np.float).mean(),decimals=2)
husband_somehi_famsize=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["famsize num only"],dtype=np.float).mean(),decimals=2)
husband_diff_famsize=np.round(husband_somehi_famsize-husband_nohi_famsize,decimals=2)
husband_diff_famsize_diff=np.round(df_og9.query("hi == 0 & fml == 0")["famsize num only"].sem()-(df_og9.query("hi == 1 & fml == 0")["famsize num only"]).sem(),decimals=2)
husband_nohi_emp=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["empl"],dtype=np.float).mean(),decimals=2)
husband_somehi_emp=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["empl"],dtype=np.float).mean(),decimals=2)
husband_diff_emp_diff=np.round(husband_somehi_emp-husband_nohi_emp,decimals=2)
husband_diff_edu_sem=np.round(df_og9.query("hi == 0 & fml == 0")["empl"].sem()-(df_og9.query("hi == 1 & fml == 0")["empl"]).sem(),decimals=2)
husband_nohi_inc=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["inc"],dtype=np.float).mean(),decimals=2)
husband_somehi_inc=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["inc"],dtype=np.float).mean(),decimals=2)
husband_diff_inc_diff=np.round(husband_somehi_inc-husband_nohi_inc,decimals=2)
husband_diff_inc_sem=np.round(df_og9.query("hi == 0 & fml == 0")["inc"].sem()-(df_og9.query("hi == 1 & fml == 0")["inc"]).sem(),decimals=2)
husband_nohi_nwhite=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["nwhite"],dtype=np.float).mean(),decimals=2)
husband_somehi_nwhite=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["nwhite"],dtype=np.float).mean(),decimals=2)
husband_diff_nwhite_diff=np.round(husband_somehi_nwhite-husband_nohi_nwhite,decimals=2)
husband_diff_nwhite_sem=np.round(df_og9.query("hi == 0 & fml == 0")["nwhite"].sem()-(df_og9.query("hi == 1 & fml == 0")["nwhite"]).sem(),decimals=2)
husband_nohi_hlth=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["hlth"],dtype=np.float).mean(),decimals=2)
husband_somehi_hlth=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["hlth"],dtype=np.float).mean(),decimals=2)
husband_nohi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 0 & fml == 0")["hlth"])),decimals=2)
husband_somehi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 1 & fml == 0")["hlth"])),decimals=2)
husband_diff_hlth_diff=np.round(husband_somehi_hlth-husband_nohi_hlth,decimals=2)
husband_diff_hlth_sem=np.round(df_og9.query("hi == 0 & fml == 0")["hlth"].sem()-(df_og9.query("hi == 1 & fml == 0")["hlth"]).sem(),decimals=2)

In [424]:
print("Husbands")
pd.DataFrame(data={' ': ["Health index", "Nonwhite","Age","Education","Family Size","Employed","Family Income","Sample Size"],
                   'Some HI (1)': ["{0} [{1}]".format(husband_somehi_hlth,husband_somehi_hlth_sd), husband_somehi_nwhite,husband_somehi_age,husband_somehi_edu,husband_somehi_famsize,husband_somehi_emp,husband_somehi_inc,husband_somehi_ss],
                  'No HI (0)': ["{0} [{1}]".format(husband_nohi_hlth,husband_nohi_hlth_sd), husband_nohi_nwhite,husband_nohi_age,husband_nohi_edu,husband_nohi_famsize,husband_nohi_emp,husband_nohi_inc,husband_nohi_ss],
                  'Difference (3)': ["{0} ({1})".format(husband_diff_hlth_diff,husband_diff_hlth_sem), "{0} ({1})".format(husband_diff_nwhite_diff,husband_diff_nwhite_sem),"{0} ({1})".format(np.round(husband_somehi_age-husband_nohi_age,decimals=2),np.round(husband_diff_age,decimals=2)),"{0} ({1})".format(husband_diff_edu_diff,husband_diff_edu),"{0} ({1})".format(husband_diff_famsize,husband_diff_famsize_diff),"{0} ({1})".format(husband_diff_emp_diff,husband_diff_edu_sem),"{0} ({1})".format(husband_diff_inc_diff,husband_diff_inc_sem)," "]})

Husbands


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.98 [0.9300000071525574],3.7 [1.0099999904632568],0.28 (0.02)
1,Nonwhite,0.2,0.19,0.01 (0.01)
2,Age,44.16,41.27,2.89 (0.12)
3,Education,14.13,11.21,2.92 (0.06)
4,Family Size,3.55,4.06,-0.51 (0.02)
5,Employed,0.92,0.85,0.07 (0.01)
6,Family Income,104002,43636,60366.42 (294.68)
7,Sample Size,7866,1529,


In [425]:
wife_somehi_ss=len(df_og9.query("hi == 1 & fml == 1"));
wife_nohi_ss=len(df_og9.query("hi == 0 & fml == 1"));
wife_somehi_age=np.round(np.mean(df_og9.query("hi == 1 & fml == 1")["age"]),decimals=2);
wife_nohi_age=np.round(np.mean(df_og9.query("hi == 0 & fml == 1")["age"]),decimals=2);
wife_diff_age=df_og9.query("hi == 0 & fml == 1")["age"].sem()-(df_og9.query("hi == 1 & fml == 1")["age"]).sem()
wife_somehi_edu=np.round(np.mean(df_og9.query("hi == 1 & fml == 1")["yedu"]),decimals=2)
wife_nohi_edu=np.round(np.mean(df_og9.query("hi == 0 & fml == 1")["yedu"]),decimals=2)
wife_diff_edu=np.round(df_og9.query("hi == 0 & fml == 1")["yedu"].sem()-(df_og9.query("hi == 1 & fml == 1")["yedu"]).sem(),decimals=2)
wife_diff_edu_diff=np.round(wife_somehi_edu-wife_nohi_edu,decimals=2)
df_og9["famsize num only"]=df_og9["famsize"].str.split(expand=True)[0]
df_og9["famsize num only"]=pd.to_numeric(df_og9["famsize num only"],errors="coerce")
wife_nohi_famsize=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["famsize num only"],dtype=np.float).mean(),decimals=2)
wife_somehi_famsize=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["famsize num only"],dtype=np.float).mean(),decimals=2)
wife_diff_famsize=np.round(wife_somehi_famsize-wife_nohi_famsize,decimals=2)
wife_diff_famsize_diff=np.round(df_og9.query("hi == 0 & fml == 1")["famsize num only"].sem()-(df_og9.query("hi == 1 & fml == 0")["famsize num only"]).sem(),decimals=2)
wife_nohi_emp=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["empl"],dtype=np.float).mean(),decimals=2)
wife_somehi_emp=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["empl"],dtype=np.float).mean(),decimals=2)
wife_diff_emp_diff=np.round(wife_somehi_emp-wife_nohi_emp,decimals=2)
wife_diff_edu_sem=np.round(df_og9.query("hi == 0 & fml == 1")["empl"].sem()-(df_og9.query("hi == 1 & fml == 0")["empl"]).sem(),decimals=2)
wife_nohi_inc=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["inc"],dtype=np.float).mean(),decimals=2)
wife_somehi_inc=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["inc"],dtype=np.float).mean(),decimals=2)
wife_diff_inc_diff=np.round(wife_somehi_inc-wife_nohi_inc,decimals=2)
wife_diff_inc_sem=np.round(df_og9.query("hi == 0 & fml == 1")["inc"].sem()-(df_og9.query("hi == 1 & fml == 1")["inc"]).sem(),decimals=2)
wife_nohi_nwhite=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["nwhite"],dtype=np.float).mean(),decimals=2)
wife_somehi_nwhite=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["nwhite"],dtype=np.float).mean(),decimals=2)
wife_diff_nwhite_diff=np.round(wife_somehi_nwhite-wife_nohi_nwhite,decimals=2)
wife_diff_nwhite_sem=np.round(df_og9.query("hi == 0 & fml == 1")["nwhite"].sem()-(df_og9.query("hi == 1 & fml == 1")["nwhite"]).sem(),decimals=2)
wife_nohi_hlth=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["hlth"],dtype=np.float).mean(),decimals=2)
wife_somehi_hlth=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["hlth"],dtype=np.float).mean(),decimals=2)
wife_nohi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 0 & fml == 1")["hlth"])),decimals=2)
wife_somehi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 1 & fml == 1")["hlth"])),decimals=2)
wife_diff_hlth_diff=np.round(wife_somehi_hlth-wife_nohi_hlth,decimals=2)
wife_diff_hlth_sem=np.round(df_og9.query("hi == 0 & fml == 1")["hlth"].sem()-(df_og9.query("hi == 1 & fml == 1")["hlth"]).sem(),decimals=2)

In [426]:
print("Wives")
pd.DataFrame(data={' ': ["Health index", "Nonwhite","Age","Education","Family Size","Employed","Family Income","Sample Size"],
                   'Some HI (1)': ["{0} [{1}]".format(wife_somehi_hlth,wife_somehi_hlth_sd), wife_somehi_nwhite,wife_somehi_age,wife_somehi_edu,wife_somehi_famsize,wife_somehi_emp,wife_somehi_inc,wife_somehi_ss],
                  'No HI (0)': ["{0} [{1}]".format(wife_nohi_hlth,wife_nohi_hlth_sd), wife_nohi_nwhite,wife_nohi_age,wife_nohi_edu,wife_nohi_famsize,wife_nohi_emp,wife_nohi_inc,wife_nohi_ss],
                  'Difference (3)': ["{0} ({1})".format(wife_diff_hlth_diff,wife_diff_hlth_sem), "{0} ({1})".format(wife_diff_nwhite_diff,wife_diff_nwhite_sem),"{0} ({1})".format(np.round(wife_somehi_age-wife_nohi_age,decimals=2),np.round(wife_diff_age,decimals=2)),"{0} ({1})".format(wife_diff_edu_diff,wife_diff_edu),"{0} ({1})".format(wife_diff_famsize,wife_diff_famsize_diff),"{0} ({1})".format(wife_diff_emp_diff,wife_diff_edu_sem),"{0} ({1})".format(wife_diff_inc_diff,wife_diff_inc_sem)," "]})

Wives


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.99 [0.9300000071525574],3.61 [1.0199999809265137],0.38 (0.02)
1,Nonwhite,0.2,0.18,0.02 (0.01)
2,Age,42.16,39.51,2.65 (0.12)
3,Education,14.28,11.36,2.92 (0.06)
4,Family Size,3.55,4.07,-0.52 (0.03)
5,Employed,0.76,0.54,0.22 (0.01)
6,Family Income,103379,43640,59738.96 (307.07)
7,Sample Size,7961,1446,
